# Исследовательский анализ данных

In [1]:
import pandas as pd
import json
import folium
import requests


import numpy as np
from shapely.geometry import LineString, shape, Point, Polygon
from shapely.ops import nearest_points
from pyproj import Transformer


Загружаем и распаковываем данные, которые мы получили из API.

In [2]:
# Запрос к серверу для получения данных последней миссии
response = requests.get("http://localhost:5005/get-mission")
mission_data = response.json()

drone_data = mission_data.get('droneData', {})
route_points = mission_data.get('routePoints', [])
saved_polygons = mission_data.get('savedPolygons', {})

print(drone_data)
print(route_points)
print(saved_polygons)

{'altitude': 270.05903056786514, 'lat': 55.139592, 'lng': 37.962471}
[{'altitude': 303.3406611600492, 'flightAltitude': 33, 'groundAltitude': 270.3406611600492, 'lat': 55.13996186227834, 'lng': 37.96202392480538}, {'altitude': 302.6175387399725, 'flightAltitude': 33, 'groundAltitude': 269.6175387399725, 'lat': 55.14092878559538, 'lng': 37.96183213192185}, {'altitude': 301.28897745043497, 'flightAltitude': 33, 'groundAltitude': 268.28897745043497, 'lat': 55.141237329274134, 'lng': 37.961940158322875}, {'altitude': 300.9808147847749, 'flightAltitude': 33, 'groundAltitude': 267.9808147847749, 'lat': 55.141031337580586, 'lng': 37.96294195376393}, {'altitude': 305.2288281562144, 'flightAltitude': 33, 'groundAltitude': 272.2288281562144, 'lat': 55.14075631273289, 'lng': 37.96393416470468}, {'altitude': 306.70394583843915, 'flightAltitude': 33, 'groundAltitude': 273.70394583843915, 'lat': 55.14038035373463, 'lng': 37.964851291875505}, {'altitude': 310.04080437566097, 'flightAltitude': 33, 'gr

Наносим на карту

In [3]:
# Создаём карту
m = folium.Map(location=[drone_data.get('lat', 0), drone_data.get('lng', 0)], zoom_start=16)

# Добавляем маркер дрона красного цвета
folium.Marker(
    [drone_data.get('lat', 0), drone_data.get('lng', 0)],
    popup='Дрон',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Добавляем маршрутные точки
for pt in route_points:
    folium.Marker(
        [pt.get('lat', 0), pt.get('lng', 0)],
        popup=f"Информация: {pt.get('lat', 'нет данных'), pt.get('lng', 'нет данных'), pt.get('altitude', 'нет данных')}",
        icon=folium.Icon(color='blue', icon='cloud')  # Можно выбрать любой цвет и иконку
    ).add_to(m)

# Соединяем маршрутные точки пунктирной линией
if len(route_points) > 1:
    folium.PolyLine(
        locations=[(pt.get('lat', 0), pt.get('lng', 0)) for pt in route_points],
        color='blue',
        weight=3,
        opacity=0.7,
        dash_array="5, 5" # пунктир
    ).add_to(m)


# Соединяем дрон с первой маршрутной точкой (если маршрут не пуст)
if route_points:
    first_point = route_points[0]
    folium.PolyLine(
        locations=[
            (drone_data.get('lat', 0), drone_data.get('lng', 0)),
            (first_point.get('lat', 0), first_point.get('lng', 0))
        ],
        color="red",
        weight=2,
        opacity=0.7,
        dash_array="5, 5"
    ).add_to(m)



# Добавляем полигоны
if saved_polygons and saved_polygons.get('features'):
    folium.GeoJson(saved_polygons).add_to(m)

# Отображаем карту
m

In [4]:
# # Создаем объект Polygon из первого элемента saved_polygons
#
# polygon_wgs = shape(saved_polygons["features"][0]["geometry"])
# # Проверяем, что тип объекта - Polygon
# assert polygon_wgs.geom_type == 'Polygon'

Преобразуем маршрутные точки в LineString и генерируем новые с шагом 1м

In [5]:
# Преобразование координат: из WGS84 в UTM
# Для Москвы часто используют UTM zone 37N (EPSG:32637), проверьте корректность для вашей области.
transformer_to_m = Transformer.from_crs("epsg:4326", "epsg:32637", always_xy=True)
transformer_to_wgs = Transformer.from_crs("epsg:32637", "epsg:4326", always_xy=True)

# Преобразуем координаты маршрутных точек в UTM
route_coords_m = []
for pt in route_points:
    x, y = transformer_to_m.transform(pt["lng"], pt["lat"])
    route_coords_m.append((x, y))

# Создаем LineString в метрической системе
route_line = LineString(route_coords_m)

# Генерируем новые точки на маршруте с шагом 1 метр
step = 1.0
points_along_route = []
distance = 0.0

while distance <= route_line.length:
    pt = route_line.interpolate(distance)
    points_along_route.append(pt)
    distance += step


In [6]:
# Преобразуем полигон в UTM (для корректных расчетов)
def transform_polygon_to_utm(polygon_wgs, transformer):
    # Получаем координаты полигона, преобразуем каждую точку
    utm_coords = []
    for ring in polygon_wgs.exterior.coords:
        x, y = transformer.transform(ring[0], ring[1])
        utm_coords.append((x, y))
    return Polygon(utm_coords)

# polygon_utm = transform_polygon_to_utm(polygon_wgs, transformer_to_m)

In [7]:
# Преобразуем каждый полигон из saved_polygons в объект Shapely и затем в UTM
polygons_utm = []
for feature in saved_polygons["features"]:
    poly_wgs = shape(feature["geometry"])
    poly_utm = transform_polygon_to_utm(poly_wgs, transformer_to_m)
    polygons_utm.append(poly_utm)

In [8]:
# Функция для вычисления проекции точки на отрезок (ребро)
def point_to_segment_projection(point, seg_start, seg_end):
    """
    point, seg_start, seg_end - кортежи (x, y)
    Возвращает: (projection, distance)
    projection: координаты проекции (x, y)
    distance: расстояние от точки до проекции
    """
    p = np.array(point)
    a = np.array(seg_start)
    b = np.array(seg_end)
    ab = b - a
    if np.allclose(ab, 0):
        return a, np.linalg.norm(p - a)
    t = np.dot(p - a, ab) / np.dot(ab, ab)
    if t < 0:
        projection = a
    elif t > 1:
        projection = b
    else:
        projection = a + t * ab
    distance = np.linalg.norm(p - projection)
    return projection, distance


In [9]:
# Функция для получения ближайшей проекции (перпендикуляра) от точки к границе полигона
def get_nearest_projection_on_polygon(point, polygon):
    """
    point: shapely Point (в UTM)
    polygon: shapely Polygon (в UTM)
    Возвращает: (projection, distance), где projection - кортеж (x, y)
    """
    coords = list(polygon.exterior.coords)
    min_distance = float('inf')
    best_proj = None
    # Проходим по каждому ребру полигона
    for i in range(len(coords) - 1):  # последний элемент равен первому
        a = coords[i]
        b = coords[i+1]
        proj, dist = point_to_segment_projection((point.x, point.y), a, b)
        if dist < min_distance:
            min_distance = dist
            best_proj = proj
    return best_proj, min_distance

In [10]:
# Функция для перебора всех полигонов и выбора проекции с минимальным расстоянием
def get_nearest_projection(point, polygons):
    overall_best_proj = None
    overall_min_dist = float('inf')
    for poly in polygons:
        proj, dist = get_nearest_projection_on_polygon(point, poly)
        if dist < overall_min_dist:
            overall_min_dist = dist
            overall_best_proj = proj
    return overall_best_proj, overall_min_dist

In [12]:
# Для каждой сгенерированной точки вдоль маршрута находим проекцию на ближайшее ребро полигона
projections = []
for pt in points_along_route:
    best_proj, d = get_nearest_projection(pt, polygons_utm)
    projections.append((pt, Point(best_proj)))

In [14]:
# Преобразуем точки и проекции обратно в WGS84 для отображения
points_along_route_wgs = []
projections_wgs = []
for pt, proj_pt in projections:
    lng_pt, lat_pt = transformer_to_wgs.transform(pt.x, pt.y)
    lng_proj, lat_proj = transformer_to_wgs.transform(proj_pt.x, proj_pt.y)
    points_along_route_wgs.append((lat_pt, lng_pt))
    projections_wgs.append((lat_proj, lng_proj))

In [15]:
# Отрисовка на карте с помощью Folium
m = folium.Map(location=[drone_data["lat"], drone_data["lng"]], zoom_start=14)

# Маркер дрона (красный)
folium.Marker(
    [drone_data["lat"], drone_data["lng"]],
    popup='Дрон',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Отобразим сгенерированные точки маршрута (синие кружки)
for pt in points_along_route_wgs:
    folium.CircleMarker(
        location=pt,
        radius=1,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(m)

# Соединяем последовательные маршрутные точки (пунктирной линией)
folium.PolyLine(
    locations=[(pt["lat"], pt["lng"]) for pt in route_points],
    color="blue",
    weight=3,
    opacity=0.7,
    dash_array="5, 5"
).add_to(m)

# Отображаем полигоны через GeoJson
folium.GeoJson(saved_polygons).add_to(m)

# Отображаем линии-перпендикуляры (зеленые) для каждой точки
for pt, proj_pt in zip(points_along_route_wgs, projections_wgs):
    folium.PolyLine(
        locations=[pt, proj_pt],
        color="green",
        weight=1,
        opacity=0.8,
        dash_array="2, 4"
    ).add_to(m)

m

Отступаем произвольное число в метрах от точки соприкосновения каждого полигона на самой проекции

In [17]:
offset_distance = 3.0  # отступ в метрах

# Предполагаем, что projections имеет формат: [(pt, proj_pt), ...]
# где pt – исходная точка (shapely.Point) вдоль маршрута (в UTM),
# а proj_pt – проекция этой точки на полигон (также shapely.Point, в UTM)

corrected_projections = []
for pt, proj_pt in projections:
    # Вычисляем вектор от проекции к исходной точке
    v = np.array([pt.x - proj_pt.x, pt.y - proj_pt.y])
    norm_v = np.linalg.norm(v)
    if norm_v == 0:
        # Если вектор нулевой, оставляем проекцию без смещения
        corrected_point = proj_pt
    else:
        unit_v = v / norm_v
        corrected_coords = np.array([proj_pt.x, proj_pt.y]) + offset_distance * unit_v
        corrected_point = Point(corrected_coords)
    corrected_projections.append(corrected_point)

In [18]:
corrected_points_wgs = []
for cp in corrected_projections:
    lng, lat = transformer_to_wgs.transform(cp.x, cp.y)
    corrected_points_wgs.append((lat, lng))

In [20]:
# Создаем карту
m = folium.Map(location=[drone_data["lat"], drone_data["lng"]], zoom_start=14)

# Отрисовываем полигоны
if saved_polygons and saved_polygons.get('features'):
    folium.GeoJson(saved_polygons, style_function=lambda x: {
        'fillColor': 'gray', 'color': 'black', 'weight': 2, 'fillOpacity': 0.3
    }).add_to(m)

# Добавляем маркер дрона
folium.Marker(
    [drone_data["lat"], drone_data["lng"]],
    popup='Дрон',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Отрисовываем исходный маршрут (например, пунктирной линией) по исходным точкам
folium.PolyLine(
    locations=[(pt["lat"], pt["lng"]) for pt in route_points],
    color="blue",
    weight=3,
    opacity=0.7,
    dash_array="5,5"
).add_to(m)

# Отображаем смещённые точки скорректированного маршрута (зелёные кружки)
for pt in corrected_points_wgs:
    folium.CircleMarker(
        location=pt,
        radius=2,
        color='green',
        fill=True,
        fill_color='green'
    ).add_to(m)

# Соединяем смещённые точки линией (скорректированная миссия)
folium.PolyLine(
    locations=corrected_points_wgs,
    color="green",
    weight=3,
    opacity=0.8
).add_to(m)

m